<a href="https://colab.research.google.com/github/Madhu-712/Knowledge_Graph_RAG/blob/main/KG_RAG_for_ecommerce_csv_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-experimental langchain-community langchain networkx langchain_google_genai

In [ ]:
!pip install --upgrade langchain-community

In [ ]:
from langchain_community.graphs.networkx_graph import NetworkxEntityGraph

In [ ]:
import os
import time
from langchain.chains import GraphQAChain
import pandas as pd
import google.generativeai as genai
#from langchain_community.graphs.networkx_graph import NetworkEntityGraph
from langchain_google_genai import GoogleGenerativeAI

In [ ]:

# Make sure you are passing the actual value of the API key,
# not the string "GOOGLE_API_KEY"
genai.configure(api_key="AIuizaSyCyFJRtCy74KGYvezEorzWt74qjkl8d85r9dc")

# Similarly, pass the actual API key value here:
llm = GoogleGenerativeAI(
    model="gemini-pro", google_api_key="AIzaSyCyFJRtCy74KGYvezEorzWklmt74q8d85r9dc"
)

# Now try invoking the LLM again:
#llm.invoke('what is knowledge graph?')

In [ ]:
llm.invoke ("what is ontology and knowledge graphs?")

'**Ontology**\n\nAn ontology is a formal representation of a domain of knowledge. It provides a shared understanding of the concepts, relationships, and axioms within that domain. Ontologies are used in various fields, including computer science, information science, and philosophy.\n\n**Components of an Ontology:**\n\n* **Concepts:** Entities or classes of entities within the domain.\n* **Relationships:** Connections between concepts, such as "has_part" or "is_a".\n* **Axioms:** Rules or statements that define the properties and behaviors of concepts and relationships.\n\n**Benefits of Ontologies:**\n\n* Improved data organization and consistency\n* Enhanced data interoperability and exchange\n* Facilitated knowledge discovery and reasoning\n* Support for artificial intelligence and machine learning\n\n**Knowledge Graphs**\n\nA knowledge graph is a large-scale, interconnected network of entities, concepts, and relationships. It represents real-world knowledge in a structured and machi

In [ ]:

import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("GOOGLE_API_KEY:\n")

In [ ]:
GOOGLE_API_KEY="AIzaSyCyFJRtCy7lkm4KGYvezEorzWt74q8d85r9dc"

In [ ]:
genai.configure(api_key='GOOGLE_API_KEY')

In [ ]:
llm=GoogleGenerativeAI(model="gemini-pro",google_api_key="GOOGLE_API_KEY")

In [ ]:
df=pd.read_csv('data.csv',encoding='latin-1')

In [ ]:
df['Quantity'].values

array([6, 6, 8, ..., 4, 4, 3])

In [ ]:
print(df['Quantity'].values.max())

74215


In [ ]:

row_data = df.iloc[74215]
print(row_data)

InvoiceNo                        542408
StockCode                         21533
Description    RETROSPOT LARGE MILK JUG
Quantity                              1
InvoiceDate             1/27/2011 16:29
UnitPrice                          4.95
CustomerID                      13280.0
Country                  United Kingdom
Name: 74215, dtype: object


In [ ]:

max_quantity_row = df.loc[df['Quantity'] == df['Quantity'].max()]
max_quantity_value = max_quantity_row['Quantity'].values[0]

print(max_quantity_value)

74215


In [ ]:

max_quantity_index = df['Quantity'].idxmax()
max_quantity_value = df.loc[max_quantity_index, 'Quantity']

print(max_quantity_value)

74215


In [ ]:

def quantity(x):
    if x<10:
       return 'bulk supply dont exist'
    else:
       return 'not bulk supply exist'
df['quantity']=df.apply(lambda x:quantity(x['Quantity']),axis=1) # Change 'quantity' to 'Quantity'

In [ ]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,quantity
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,bulk supply dont exist
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,bulk supply dont exist
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,bulk supply dont exist
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,bulk supply dont exist
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,bulk supply dont exist
...,...,...,...,...,...,...,...,...,...
566681,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,not bulk supply exist
566682,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,bulk supply dont exist
566683,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,bulk supply dont exist
566684,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,bulk supply dont exist


In [ ]:

graph=NetworkxEntityGraph()

for id,row in df.iterrows():
     graph.add_node(row['InvoiceNo'])
     graph.add_node(row['Description'])
     #graph.add_node(row['UnitPrice'])
for id,row in df.iterrows():
     # The correct method to add edges is 'add_edge', not 'add_edges'
     graph._graph.add_edge(row['InvoiceNo'],
                           row['Description'],
                           #row['UnitPrice'],
                           relation=row['Quantity'])
                           # relation=row['UnitPrice'])

In [ ]:
chain=GraphQAChain.from_llm(
         llm=llm,
         graph=graph,
         verbose=True)

In [ ]:

question = "What is the description of the InvoiceNo 536365?"
chain.run(question)



> Entering new GraphQAChain chain...
Entities Extracted:
InvoiceNo, 536365
Full Context:
536365 6 WHITE HANGING HEART T-LIGHT HOLDER
536365 6 WHITE METAL LANTERN
536365 8 CREAM CUPID HEARTS COAT HANGER
536365 6 KNITTED UNION FLAG HOT WATER BOTTLE
536365 6 RED WOOLLY HOTTIE WHITE HEART.
536365 2 SET 7 BABUSHKA NESTING BOXES
536365 6 GLASS STAR FROSTED T-LIGHT HOLDER

> Finished chain.


"I don't know."

In [ ]:

from langchain_community.graphs.networkx_graph import NetworkxEntityGraph

graph = NetworkxEntityGraph()

# Add nodes for Invoice Numbers, Descriptions, and potentially Unit Prices
for _, row in df.iterrows():
    graph.add_node(row['InvoiceNo']),# entity_type="InvoiceNo")
    graph.add_node(row['Description']), #entity_type="Description")
    # If you need to query Unit Prices, add a node for them as well
    graph.add_node(row['UnitPrice']), #entity_type="UnitPrice")

# Add edges to represent relationships
for _, row in df.iterrows():
    graph._graph.add_edge(row['InvoiceNo'], row['Description'], relation="has_description")
    # Add an edge for Unit Price if needed
    graph._graph.add_edge(row['InvoiceNo'], row['UnitPrice'], relation="has_unitprice")
    # or
    graph._graph.add_edge(row['Description'], row['UnitPrice'], relation="has_unitprice")

# Create the GraphQAChain and run the query
chain = GraphQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True
)

question = "Are there any descriptions included for InvoiceNo 581587?"
chain.run(question)



> Entering new GraphQAChain chain...
Entities Extracted:
NONE
Full Context:


> Finished chain.


"I don't know."